# Example of building and training a convolutional Neural Network
In this example, a convolutional neural netowork is created using the `neural` framework. This network is than trained on the MNIST data set of hand-written digits.  
**Note:** The training of this network lasts around half an hour depending on your machine. By default results are shown for a pretrained network.
You can still train your own network by setting the `usePretrained` variable to `False`.

In [ ]:
usePretrained = True

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import sys
sys.path.append("..")

In [ ]:
import numpy as np
import time

from neural import MNIST, Tensor, nn, optim
from utils import *

## Importing MNIST training data

In [ ]:
# Loading training set
allTrainImages, allTrainLabels = MNIST.get("train")
# Images are normalized, all values are in the range [-1, 1]
allTrainImages = normalize(allTrainImages, 0.5, 0.5)

## Defining the Neural Network architecture

In [ ]:
class Network(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 3, 5)
        self.conv2 = nn.Conv2d(3, 6, 5)
        self.fc1 = nn.Linear(384, 60)
        self.fc2 = nn.Linear(60, 10)
        self.dropout = nn.Dropout(p=0.2)
        self.pool = nn.MaxPool2d(2, stride=2)
        
    def forward(self, x):
        x = self.dropout(self.pool(nn.ReLU()(self.conv1(x))))
        x = nn.ReLU()(self.conv2(x))
        x.reshape_((x.shape[0], -1))
        x = nn.ReLU()(self.fc1(x))
        x = self.fc2(x)
        return x
    
model = Network()

## Choosing training criterion (loss function) and optimizer

In [ ]:
# Loss function
reduction = "mean"
criterion = nn.CrossEntropyLoss(reduction=reduction)

# SGD parameters
lr = 0.03
momentum = 0.9
weightDecay = 0.0
dampening = 0.0
nesterov = False

optimizer = optim.SGD(
    model.parameters(),
    lr=lr,
    momentum=momentum,
    dampening=dampening,
    weightDecay=weightDecay,
    nesterov=nesterov)

## Training

### Choosing training parameters

In [ ]:
epochs = 1
batchSize = 100

In [ ]:
numBatches = allTrainImages.shape[0] // batchSize
numTraining = int(numBatches * batchSize)

print(f"Number of epochs: {epochs}")
print(f"Batch size: {batchSize}")
print(f"Total number of train images: {numTraining}")
print(f"Total number of batches: {numBatches}")

# Reshaping training data
trainImages = allTrainImages[:numTraining].reshape(numBatches, -1, allTrainImages.shape[-2], allTrainImages.shape[-1])
trainLabels = allTrainLabels[:numTraining].reshape(numBatches, -1)

### Running epochs

In [ ]:
if not usePretrained:
    lossTrack = np.zeros((epochs, numBatches))
    for e in range(epochs):
        startTime = time.time()
        for i, (images, labels) in enumerate(zip(trainImages, trainLabels)):
            images = images[:, None, :, :] # Adding dummy axis to serve as number of channels
            optimizer.zeroGrad()
            out = model(images)
            loss = criterion(out, labels)
            loss.backward()
            optimizer.step()
            lossTrack[e, i] = loss.item()
        else:
            endTime = time.time()
            print(f"Finished epoch {e} in {endTime - startTime:.2f}s")
else:
    model = nn.Module.load("convolutional.pkl")

In [ ]:
if not usePretrained:
    plotLossTrack([lossTrack], [f"Learning rate: {lr}\nMomentum: {momentum}\nBatch size: {batchSize}"])

### Saving the trained model

In [ ]:
nn.Module.save(model, "convolutional.pkl")

Saved module can be loaded with
```python
model = nn.Module.load("convolutional.pkl")
```

## Performance evaluation

In [ ]:
# Image iterator
imgIter = iter(allTrainImages)

Run the cell bellow multiple times to check model performance for different images.

In [ ]:
img = next(imgIter)
img_ = img[None, :, :]

scores = model(img_)
ps = nn.softmax(scores)

showMNIST(img.squeeze(), ps.squeeze())